In [ ]:
import os
import pymysql
import datetime

In [ ]:
conda uninstall pandas

In [ ]:
os.environ['PACKAGE_HOME']="/Users/bcard/OneDrive - American Institutes for Research in the Behavioral Sciences/projects/patentsview/PatentsView-DB"


In [ ]:
cd ..

In [ ]:
from lib.configuration import get_config, get_connection_string, get_current_config


c=get_config()

In [ ]:
config = get_current_config('pgpubs', **{
        "execution_date": datetime.date(2021, 11, 4)
    })

In [ ]:
connection_string=get_connection_string(config, database="PROD_DB")

In [ ]:
connection_string

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine=create_engine(connection_string)

In [ ]:
from pandas import read_sql_query

In [ ]:
# columns=pd.read_sql_query(sql="""
# select *
# from information_schema.COLUMNS
# where TABLE_SCHEMA = 'pregrant_publications'
#   and TABLE_NAME not like '%%temp%%'
#   and TABLE_NAME not like '%%ernest%%'
#   and TABLE_NAME not like 'brf_sum_text%%'
#   and TABLE_NAME not like 'detail_desc%%'
#   and table_name not like 'draw_desc%%'
#   and table_name not like 'claim_%%' group by TABLE_NAME;
# """, con=engine)

In [ ]:
columns=pd.read_sql_query(sql="""
select *
from information_schema.COLUMNS
where TABLE_SCHEMA = 'pregrant_publications'
  and TABLE_NAME not like '%%tmp%%'
  and TABLE_NAME not like '%%temp%%'
  and TABLE_NAME not like '%%backup%%'
  and TABLE_NAME not like '%%20201229%%'
  and TABLE_NAME not like '%%20201231%%'
  and TABLE_NAME not like '%%20210330%%'
  and TABLE_NAME not like '%%20210629%%'
  and TABLE_NAME not like '%%20210930%%'
  and TABLE_NAME not like '%%archive%%'
  and TABLE_NAME not like '%%disamb%%'
  and TABLE_NAME not like 'brf_sum_text_%%'
  and TABLE_NAME not like 'detail_desc_text_%%'
  and TABLE_NAME not like 'claim_%%'
  and TABLE_NAME not like 'draw_desc_text_%%'
""", con=engine)

In [ ]:
columns.groupby('TABLE_NAME').get_group('application')

In [ ]:
columns['TABLE_NAME'].unique()

In [ ]:
## Pulled From DataMonitor_categorycount JSON
category_list = ['category',
'class',
'class_data_source',
'class_status',
'class_value',
#'country', 
'deceased',
'designation',
'doc_type',
'filing_type',
'kind',
'main_group',
'mainclass_id',
'relkind',
'section',
'subclass',
'symbol_position',
'type',
'value',
'version']


In [ ]:
def generate_column_setting(column):
    
    data_type = 'varchar'
    if column.DATA_TYPE in ['date', 'timestamp']:
        data_type = 'date'
    if column.DATA_TYPE in ['int', 'bigint', 'smallint', 'float', 'decimal']:
        data_type = 'int'
    
    null_allowed = True
    if column.IS_NULLABLE == 'NO':
        null_allowed = False
    
    location_field = False
    if column.COLUMN_NAME in ['country','country_transformedd']:
        location_field = True
    
    category = False
    if column.COLUMN_NAME in category_list:
        if column.TABLE_NAME in ['further_cpc','main_cpc','ipcr']:
            print("Skipping this one")
        else:
            category=True
        
    setting = {
        column.COLUMN_NAME: {
            'data_type': data_type,
            'null_allowed': null_allowed,
            'category': category,
            'location_field': location_field
        }
    }
    return setting

In [ ]:
from functools import reduce


In [ ]:
def generate_table_setting(table_group):
    y=table_group.apply(generate_column_setting,axis=1)
    t_set={table_group.name:{"fields":reduce(lambda a, b: dict(a, **b), y.tolist())}}
    return t_set

In [ ]:
x=columns.groupby('TABLE_NAME').apply(func=generate_table_setting)

In [ ]:
x.keys()

In [ ]:
c_set=reduce(lambda a, b: dict(a, **b), x.tolist())

In [ ]:
c_set

In [ ]:
foreign_tables = []
for i in c_set.keys():
#     print(i)
    if i != 'publication':
        for j in c_set[i]['fields']:
    #         print("=====", j)
            if j == 'document_number':
                foreign_tables.append(['publication', 'document_number', i, 'document_number'])

In [ ]:
related_entities_lookup1 = pd.DataFrame(list(foreign_tables), columns=['main_table','main_table_id','related_table','related_table_id'])


In [ ]:
related_entities_lookup1

In [ ]:
foreign_tables = []
for i in c_set.keys():
#     print(i)
    for j in c_set[i]['fields']:
#         print("=====", j)
        words = j.split("_")
        for w in words:
            if w in c_set.keys():
                if i != w:
                    if j=='application_number':
                        dest_id = 'application_number'
#                     elif w[:3] == 'raw':
#                         dest_id = 'uuid'
                    else:
                        dest_id = 'id'
                    foreign_tables.append([i, j, w, dest_id])

In [ ]:
related_entities_lookup2 = pd.DataFrame(list(foreign_tables), columns=['main_table','main_table_id','related_table','related_table_id'])


In [ ]:
related_entities_lookup2

In [ ]:
related_entities_lookup = pd.concat([related_entities_lookup1, related_entities_lookup2])


In [ ]:
# ADDING IN RELATIONAL ENTITIES 
for i in c_set.keys():
    if i in set(list(related_entities_lookup["main_table"])):
        related_tables = list(related_entities_lookup["related_table"][related_entities_lookup["main_table"]==i])
        dict_list = []
        for z in related_tables:
            table = related_entities_lookup["related_table"][ (related_entities_lookup["main_table"]==i) & (related_entities_lookup["related_table"]==z) ].item()
            mt_id = related_entities_lookup["main_table_id"][ (related_entities_lookup["main_table"]==i) & (related_entities_lookup["related_table"]==z) ].item()
            rel_id = related_entities_lookup["related_table_id"][ (related_entities_lookup["main_table"]==i) & (related_entities_lookup["related_table"]==z) ].item()
            temp = {
                "related_table": table,
                "main_table_id": mt_id,
                "related_table_id": rel_id
                }
            dict_list.append(temp)
        c_set[i]['related_entities']=dict_list
            

In [ ]:
c_set['rawassignee']['related_entities']

In [ ]:
# PRINT FUNCTION FOR COPYING THE OUTPUT INTO JSON
c_set

In [ ]:
exclusion_list = []
for i in c_set.keys():
    print(i,"TABLE =====================")
    temp = 0
    for j in c_set[i]['fields']:
        print(j, "FIELD")
        if j == 'document_number':
            temp += 1
        else:
            pass
    print(temp)
    if temp == 1:
        pass
    else:
        exclusion_list.append(i)
        
    

In [ ]:
exclusion_list

In [ ]:
for related_entity_config in c_set['publication_assignee']['related_entities']:
    print(related_entity_config)
    

In [ ]:
c_set.keys()

In [ ]:
test_script_list = []
for i in c_set.keys():
    if i in MergeTest:
        test_script_list.append("MergeTestWeekly")
        test_script_list.append("MergeTestQuarterly")
    if i in UploadTest:
        test_script_list.append("UploadTest")
#     if i[:7] in ['brf_sum','claims_','draw_de','detail_']:
#         test_script_list.append("TextTest")
    c_set[i]['TestScripts'] = test_script_list
    test_script_list = []

In [ ]:
c_set

In [ ]:
UploadTest = ['application',
'brf_sum_text',
'brf_sum_text_2001',
'brf_sum_text_2002',
'brf_sum_text_2003',
'brf_sum_text_2004',
'brf_sum_text_2005',
'brf_sum_text_2006',
'brf_sum_text_2007',
'brf_sum_text_2008',
'brf_sum_text_2009',
'brf_sum_text_2010',
'brf_sum_text_2011',
'brf_sum_text_2012',
'brf_sum_text_2013',
'brf_sum_text_2014',
'brf_sum_text_2015',
'brf_sum_text_2016',
'brf_sum_text_2017',
'brf_sum_text_2018',
'brf_sum_text_2019',
'brf_sum_text_2020',
'brf_sum_text_2021',
'claim',
'claim_2001',
'claim_2002',
'claim_2003',
'claim_2004',
'claim_2005',
'claim_2006',
'claim_2007',
'claim_2008',
'claim_2009',
'claim_2010',
'claim_2011',
'claim_2012',
'claim_2013',
'claim_2014',
'claim_2015',
'claim_2016',
'claim_2017',
'claim_2018',
'claim_2019',
'claim_2020',
'claim_2021',
'cpc',
'detail_desc_text',
'detail_desc_text_2001',
'detail_desc_text_2002',
'detail_desc_text_2003',
'detail_desc_text_2004',
'detail_desc_text_2005',
'detail_desc_text_2006',
'detail_desc_text_2007',
'detail_desc_text_2008',
'detail_desc_text_2009',
'detail_desc_text_2010',
'detail_desc_text_2011',
'detail_desc_text_2012',
'detail_desc_text_2013',
'detail_desc_text_2014',
'detail_desc_text_2015',
'detail_desc_text_2016',
'detail_desc_text_2017',
'detail_desc_text_2018',
'detail_desc_text_2019',
'detail_desc_text_2020',
'detail_desc_text_2021',
'draw_desc_text',
'draw_desc_text_2001',
'draw_desc_text_2002',
'draw_desc_text_2003',
'draw_desc_text_2004',
'draw_desc_text_2005',
'draw_desc_text_2006',
'draw_desc_text_2007',
'draw_desc_text_2008',
'draw_desc_text_2009',
'draw_desc_text_2010',
'draw_desc_text_2011',
'draw_desc_text_2012',
'draw_desc_text_2013',
'draw_desc_text_2014',
'draw_desc_text_2015',
'draw_desc_text_2016',
'draw_desc_text_2017',
'draw_desc_text_2018',
'draw_desc_text_2019',
'draw_desc_text_2020',
'draw_desc_text_2021',
'foreign_priority',
'further_cpc',
'ipcr',
'main_cpc',
'pct_data',
'publication',
'rawassignee',
'rawinventor',
'rawlocation',
'rawuspc',
'rel_app_text',
'us_parties',
'uspc',
'usreldoc',
'usreldoc_parent_child',
'usreldoc_related',
'usreldoc_single']

In [ ]:
MergeTest = ['application',
'brf_sum_text_2001',
'brf_sum_text_2002',
'brf_sum_text_2003',
'brf_sum_text_2004',
'brf_sum_text_2005',
'brf_sum_text_2006',
'brf_sum_text_2007',
'brf_sum_text_2008',
'brf_sum_text_2009',
'brf_sum_text_2010',
'brf_sum_text_2011',
'brf_sum_text_2012',
'brf_sum_text_2013',
'brf_sum_text_2014',
'brf_sum_text_2015',
'brf_sum_text_2016',
'brf_sum_text_2017',
'brf_sum_text_2018',
'brf_sum_text_2019',
'brf_sum_text_2020',
'brf_sum_text_2021',
'claim_2001',
'claim_2002',
'claim_2003',
'claim_2004',
'claim_2005',
'claim_2006',
'claim_2007',
'claim_2008',
'claim_2009',
'claim_2010',
'claim_2011',
'claim_2012',
'claim_2013',
'claim_2014',
'claim_2015',
'claim_2016',
'claim_2017',
'claim_2018',
'claim_2019',
'claim_2020',
'claim_2021',
'cpc',
'detail_desc_text_2001',
'detail_desc_text_2002',
'detail_desc_text_2003',
'detail_desc_text_2004',
'detail_desc_text_2005',
'detail_desc_text_2006',
'detail_desc_text_2007',
'detail_desc_text_2008',
'detail_desc_text_2009',
'detail_desc_text_2010',
'detail_desc_text_2011',
'detail_desc_text_2012',
'detail_desc_text_2013',
'detail_desc_text_2014',
'detail_desc_text_2015',
'detail_desc_text_2016',
'detail_desc_text_2017',
'detail_desc_text_2018',
'detail_desc_text_2019',
'detail_desc_text_2020',
'detail_desc_text_2021',
'draw_desc_text_2001',
'draw_desc_text_2002',
'draw_desc_text_2003',
'draw_desc_text_2004',
'draw_desc_text_2005',
'draw_desc_text_2006',
'draw_desc_text_2007',
'draw_desc_text_2008',
'draw_desc_text_2009',
'draw_desc_text_2010',
'draw_desc_text_2011',
'draw_desc_text_2012',
'draw_desc_text_2013',
'draw_desc_text_2014',
'draw_desc_text_2015',
'draw_desc_text_2016',
'draw_desc_text_2017',
'draw_desc_text_2018',
'draw_desc_text_2019',
'draw_desc_text_2020',
'draw_desc_text_2021',
'foreign_priority',
'further_cpc',
'ipcr',
'main_cpc',
'pct_data',
'publication',
'rawassignee',
'rawinventor',
'rawlocation',
'rawuspc',
'rel_app_text',
'us_parties',
'uspc',
'usreldoc']

#### REMOVE UNIMPORTANT COLUMNS

In [ ]:
for i in c_set.keys():
    for j in list(c_set[i]['fields'].keys()):
        if j in ['created_date','updated_date','version_indicator','location_id_transformed']:
            del c_set[i]['fields'][j]

In [ ]:
c_set.keys()

In [ ]:
len(list(c_set.keys()) )

In [ ]:
MergeTest = ['application','claim','cpc','draw_desc_text','foreign_priority','further_cpc','ipcr','main_cpc','publication']




In [ ]:
# def without_keys(d, keys):
#     return {x: d[x] for x in d if x not in keys}

In [ ]:
# final = without_keys(c_set, exclusion_list)
# final

In [ ]:
#### UPDATES NEEDED IN REPO
# change True and False to lowercase
# change datatype to data_type